### Описание:
Классифицировать игроков в каждом кадре по двум командам на основе цвета их формы.

### Данные:
Скачайте архив с данными [отсюда](https://disk.yandex.ru/d/NMlVHytktlzbEw) и распакуйте. Внутри архива лежит папка `frames` и файл `bboxes.json`. Файл `bboxes.json` содержит bounding boxes игроков двух команд, а в папке `frames` расположены соответствующие им изображения (кадры). Всего дано 100 кадров. Каждому кадру соответствует 10 bounding boxes игроков.

Структура файла `bboxes.json` следующая:
```
frame_n: {
     player_1: {
             'box': [x,y,w,h],
             'team': int
     }
     .....
     plyaer_10: { .... }
}
....
frame_n+k: { .... }
```
Здесь `frame_n` - номер кадра (ему соответствует файл с изображением в папке frames, всего 100),<br>
`player_1` - id игрока в текущем кадре,<br>
`'box': [x,y,w,h]` - bounding box соответствующего игрока (координаты нормализованы от 0 до 1, поэтому их нужно домножить на размеры изображения, чтобы перевести их в целые координаты фрейма),<br>
`'team': int` - id команды к которой принадлежит игрок  в данном кадре (может принимать значения 0 или 1).
Такм образом, id игроков и id команд имеют смысл только в пределах одного кадра и не связаны с другими кадрами. Т.е. в разных кадрах одному и тому же игроку / команде может соответствовать разные id.

### Задачи:
1) Классифицируйте игроков, используя в качестве вектора признаков игрока (features) средний цвет в пространстве RGB, который берется из области изображения, покрытой bounding box этого игрока (т.е. вектор признаков будет состоять из 3 элементов `[r,g,b]`.<br>
2) Используйте в качестве вектора признаков игроков их средний цвет в пространстве HSV (аналогично вектор признаков `[h,s,v]`.<br>
3) Используйте в качестве вектора признаков гистограмму в пространстве RGB, а не средний цвет. Число бинов определите самостоятельно.<br>
4) Используйте в качестве вектора признаков гистограмму в пространстве HSV.

### Как классифицировать:
В качестве классификатора можете использовать:
- Методы без обучения: k-means (на 2 кластера) или другие методы кластеризации.
- Методы с обучением: K-Nearest Neighbors, RandomForest, GaussianNB или др.

### Результаты:
1. Посчитайте точность классификации для каждого варианта и сравните результаты. 
2. Задание будет принято, если итоговая точность будет выше чем 0.5
3. [опционально] При тестировании на закрытой выборке в конце курса будет подведен итог (рекорд по всем курсам на данный момент - ***0.903***).

#### Полезные советы
- Боксы игроков имеют разный размер, поэтому нормализуйте гистограмму перед ее использованием в качестве вектора признаков (делите значения гистограммы на площадь бокса). 
- Большое число бинов гистограммы может привести к худшим результатам, поэтому попробуйте использовать 8/16/32.
- Для построения вектора признаков лучше использовать некоторую его центральную область, т.к. она покрывает игрока, а границы часто содержут фон / других игроков.

## Тестирование на закрытой выборке (опционально)

Скачайте архив с тестовыми данными [отсюда](https://disk.yandex.ru/d/ZbHuDyif9PJuuA). Внутри архива 100 изображений в папке `frames` (из той же игры), и файл `bboxes_test.json`, который содержит только `bounding boxes` игроков (и не содержит полей `'team'`).

Используйте вашу лучшую модель классификатора на эти данных. В качестве результата загрузите в ЛК файл `test_results.json` который должен содержать следующие данные:
```
frame_n: {
     player_1: team_id,
     .....
     plyaer_10: team_id
},
....
frame_n+k: { .... }
``` 
где `team_id` - метка команды (0 или 1).

После этого преподаватель сообщит вам accuracy вашей модели на тестовых данных.

## Как посчитать accuracy

In [ ]:
def compare(vec1, vec2):
    res = [int(e1 == e2) for e1, e2 in zip(vec1, vec2)]
    return res

# GT:
players = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
gt_team1 = [0, 2, 4, 8, 9]
gt_team2 = [1, 3, 5, 6, 7]

# Predictions:
team1 = [0, 2, 3, 5, 9]
team2 = [1, 4, 8, 6, 7]

# Calculate accuracy:
res1 = compare(team1, gt_team1)
res2 = compare(team2, gt_team2)
if sum(res1) / len(res1) < 0.5:
    res1 = compare(team1, gt_team2)
    res2 = compare(team2, gt_team1)

accuracy = (sum(res1) / len(res1) + sum(res2) / len(res2)) * 0.5
print(res1, res2, 'accuracy:', accuracy)

[1, 1, 0, 0, 1] [1, 0, 0, 1, 1] accuracy: 0.6
